In [1]:
%%sh
cat <<EOF > kubernetes.repo
[kubernetes]
name=Kubernetes
baseurl=https://packages.cloud.google.com/yum/repos/kubernetes-el7-x86_64
enabled=1
gpgcheck=1
repo_gpgcheck=1
gpgkey=https://packages.cloud.google.com/yum/doc/yum-key.gpg https://packages.cloud.google.com/yum/doc/rpm-package-key.gpg
exclude=kube*
EOF


In [4]:
!sudo -S cp kubernetes.repo /etc/yum.repos.d/kubernetes.repo < $HOME/my_password_file

[sudo] password for mapr: 

In [9]:
!sudo -S clush -ac /etc/yum.repos.d/kubernetes.repo < $HOME/my_password_file

[sudo] password for mapr: 

In [11]:
!sudo -S clush -aB "setenforce 0" < $HOME/my_password_file

[sudo] password for mapr: ---------------
10.20.30.[66-68] (3)
---------------
setenforce: SELinux is disabled
clush: 10.20.30.[66-68] (3): exited with exit code 1


In [12]:
!sudo -S clush -aB "sed -i 's/^SELINUX=enforcing$/SELINUX=permissive/' /etc/selinux/config" < $HOME/my_password_file
!sudo -S clush -aB "yum install -y kubelet kubeadm kubectl --disableexcludes=kubernetes" < $HOME/my_password_file

---------------    : [sudo] password for mapr: clush: 0/3 write: 2 B/s
10.20.30.66
---------------
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: sjc.edge.kernel.org
 * epel: mirror.sjc02.svwh.net
 * extras: mirror.web-ster.com
 * nux-dextop: mirror.li.nux.ro
 * updates: repo.ialab.dsu.edu
Package kubelet-1.14.1-0.x86_64 already installed and latest version
Package kubeadm-1.14.1-0.x86_64 already installed and latest version
Package kubectl-1.14.1-0.x86_64 already installed and latest version
Nothing to do
---------------
10.20.30.67
---------------
Loaded plugins: fastestmirror
Loading mirror speeds from cached hostfile
 * base: sjc.edge.kernel.org
 * epel: mirror.sfo12.us.leaseweb.net
 * extras: mirror.web-ster.com
 * updates: mirrors.xmission.com
Package kubelet-1.14.1-0.x86_64 already installed and latest version
Package kubeadm-1.14.1-0.x86_64 already installed and latest version
Package kubectl-1.14.1-0.x86_64 already installed and latest versio

In [13]:
!sudo -S systemctl enable --now kubelet < $HOME/my_password_file

[sudo] password for mapr: Created symlink from /etc/systemd/system/multi-user.target.wants/kubelet.service to /usr/lib/systemd/system/kubelet.service.


In [16]:
#installing a pod network - necessary step for kubeadm
#pass ipv4 traffic to iptables' chains
!sudo -S clush -aB sysctl net.bridge.bridge-nf-call-iptables=1 < $HOME/my_password_file

[sudo] password for mapr: ---------------
10.20.30.[66-68] (3)
---------------
net.bridge.bridge-nf-call-iptables = 1


In [28]:
%%sh
sudo -S kubeadm init < $HOME/my_password_file
mkdir -p $HOME/.kube
sudo -S cp -f /etc/kubernetes/admin.conf $HOME/.kube/config < $HOME/my_password_file
sudo -S chown $(id -u):$(id -g) $HOME/.kube/config < $HOME/my_password_file
kubectl apply -f https://raw.githubusercontent.com/cloudnativelabs/kube-router/master/daemonset/kubeadm-kuberouter.yaml


[init] Using Kubernetes version: v1.14.1
[preflight] Running pre-flight checks
[preflight] Pulling images required for setting up a Kubernetes cluster
[preflight] This might take a minute or two, depending on the speed of your internet connection
[preflight] You can also perform this action in beforehand using 'kubeadm config images pull'
[kubelet-start] Writing kubelet environment file with flags to file "/var/lib/kubelet/kubeadm-flags.env"
[kubelet-start] Writing kubelet configuration to file "/var/lib/kubelet/config.yaml"
[kubelet-start] Activating the kubelet service
[certs] Using certificateDir folder "/etc/kubernetes/pki"
[certs] Generating "etcd/ca" certificate and key
[certs] Generating "etcd/server" certificate and key
[certs] etcd/server serving cert is signed for DNS names [psnode66.ps.lab localhost] and IPs [10.20.30.66 127.0.0.1 ::1]
[certs] Generating "apiserver-etcd-client" certificate and key
[certs] Generating "etcd/peer" certificate and key
[certs] etcd/peer serving c

[sudo] password for mapr: 

In [30]:
!sudo -S clush -aB systemctl enable docker.service < $HOME/my_password_file
!sudo -S clush -B -g except_current systemctl enable kubelet.service < $HOME/my_password_file
!sudo -S clush -B -g except_current kubeadm join 10.20.30.66:6443 --token 2ddhcj.uuqqj74jxc5kt5o4 \
    --discovery-token-ca-cert-hash sha256:add4bddb1def9394bdc039d64eb767cf01a231650acddb62e895ce807030167c < $HOME/my_password_file


[sudo] password for mapr: [sudo] password for mapr: ---------------
10.20.30.[67-68] (2)
---------------
Created symlink from /etc/systemd/system/multi-user.target.wants/kubelet.service to /usr/lib/systemd/system/kubelet.service.
[sudo] password for mapr: ---------------
10.20.30.[67-68] (2)
---------------
[preflight] Running pre-flight checks
error execution phase preflight: [preflight] Some fatal errors occurred:
	[ERROR FileAvailable--etc-kubernetes-kubelet.conf]: /etc/kubernetes/kubelet.conf already exists
	[ERROR FileAvailable--etc-kubernetes-bootstrap-kubelet.conf]: /etc/kubernetes/bootstrap-kubelet.conf already exists
	[ERROR Port-10250]: Port 10250 is in use
	[ERROR FileAvailable--etc-kubernetes-pki-ca.crt]: /etc/kubernetes/pki/ca.crt already exists
[preflight] If you know what you are doing, you can make a check non-fatal with `--ignore-preflight-errors=...`
clush: 10.20.30.[67-68] (2): exited with exit code 1


In [31]:
!kubectl get nodes

NAME              STATUS   ROLES    AGE   VERSION
psnode66.ps.lab   Ready    master   13m   v1.14.1
psnode67.ps.lab   Ready    <none>   19s   v1.14.1
psnode68.ps.lab   Ready    <none>   20s   v1.14.1
